## Set the enviroments.

#### Connect to slime.

In [1]:
(length (core:bformat nil "%16.8e" 1234.5676))

16

In [1]:
(start-swank)

T

#### Add paths.

In [2]:
(leap.core::clear-path)
(add-path *default-pathname-defaults*)
(add-path "~/Development/amber18/dat/leap/cmd/")
(add-path "~/Development/amber18/dat/leap/lib/")
(add-path "~/Development/amber18/dat/leap/parm/")
(add-path "~/Development/amber18/dat/leap/prep/")
(add-path "~/Development/amber18/dat/antechamber/")

(#P"/Users/shiho/Development/amber18/dat/antechamber/"
 #P"/Users/shiho/Development/amber18/dat/leap/prep/"
 #P"/Users/shiho/Development/amber18/dat/leap/parm/"
 #P"/Users/shiho/Development/amber18/dat/leap/lib/"
 #P"/Users/shiho/Development/amber18/dat/leap/cmd/"
 #P"/Users/shiho/Development/cando/extensions/cando/src/tests/alanine-demo/")

#### Load the amber force field.

In [3]:
(leap.core::clear-force-field)

NIL

In [4]:
(load-atom-type-rules "ATOMTYPE_AMBER.DEF")

(#<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [5]:
(source "leaprc.protein.ff14SB")


add-atom-types doesn't do anything

Warning!  Skipping 10-12 hbond


Warning!  Skipping force field atom equivalences



T

In [6]:
(load-atom-type-rules "water_amber.def")

(#<FORCE-FIELD water_amber.def> #<FORCE-FIELD frcmod.ff14SB>
 #<FORCE-FIELD parm10.dat> #<FORCE-FIELD ATOMTYPE_AMBER.DEF>)

In [7]:
(:= *ff* (leap.core::merged-force-field))

*FF*

## Create topology and coordinate file for Alanine

#### Load the alanine from Chemdraw file.

In [9]:
(defparameter *chemagg* (cando:load-chem-draw-aggregate "alanine2.cdxml"))

*CHEMAGG*

In [10]:
(build-hydrogens *chemagg*)

NIL

In [11]:
(cando::show-aggregate *chemagg*)


An error occurred of type: UNDEFINED-FUNCTION: 
  The function CANDO::SHOW-AGGREGATE is undefined.

serious-condition backtrace:
0: (ERROR UNDEFINED-FUNCTION :NAME CANDO::SHOW-AGGREGATE)
1: (LAMBDA)



NIL

In [12]:
(show *chemagg*)

#<NGLV:NGLWIDGET>

#### Gather atoms to the center.

In [11]:
(progn
  (defparameter *stereocenters*
    (sort (cando:gather-stereocenters *chemagg*) #'string< :key #'chem:get-name))
  (cando:set-stereoisomer-func *stereocenters* (constantly :S) :show t)
  (let ((quat-matcher (core:make-cxx-object 'chem:chem-info)))
    (chem:compile-smarts quat-matcher "[C&H0&D4]")
    (chem:map-atoms nil (lambda (a) (when (chem:matches quat-matcher a)
                  (chem:set-configuration a :S)
                  (format t "Set atom ~a to :S~%" (chem:get-name a))))
            *chemagg*)))

Reader error: Cannot find the external symbol CHEM-INFO in #<PACKAGE CHEM> (condition of type SIMPLE-PROGRAM-ERROR)


NIL

In [ ]:
(:= *chemaagg* (load-mol2 "alanin-leap.mol2"))

#### Set the bounding box.

#### Assign atom types and change one of them for alanine atom type definition.

In [10]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

In [11]:
(apropos "assign-atom-types")

ASSIGN-ATOM-TYPES  Function


NIL

In [12]:
(chem:map-atoms 'nil (lambda (a)  (format t "type ~a name ~a~% " (chem:get-type a) (chem:get-name a)))
                             *chemagg*)

type O name O
 type C name C
 type CT name C_0
 type N name NH
 type H1 name H
 type CT name CH3
 type HC name HH31
 type HC name HH32
 type HC name HH33
 type H name HH
 type O name O
 type C name C
 type CT name CH3
 type HC name HH31
 type HC name HH32
 type HC name HH33
 type N name NH
 type CT name CH3
 type H1 name HH31
 type H1 name HH32
 type H1 name HH33
 type H name HH
 

NIL

In [13]:

(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-name a) :C_0)
                                     (format t "type ~a name ~a " (chem:get-type a) (chem:get-name a))))
                             *chemagg*)

Reader error: At

  
(chem:map-atoms 'nil (la
  ^ (Line 2, Column 0, Position 1)

In context LEAP:

While parsing VARIABLE-NAME. Expected:

     a character in [a-z] or [A-Z] or [0-9] or [.] or [_]

While parsing FUNCTION-NAME-STRING. Expected:

     a character in [a-z] or [A-Z] or [0-9]

While parsing SHELL-STYLE-COMMENT. Expected:

     the character # (NUMBER_SIGN)

While parsing SPACOID-CHARACTER. Expected:

     the character Tab
  or the character Space

While parsing NEWLINE. Expected:

     the character Newline

While parsing TOKEN-;. Expected:

     the character ; (SEMICOLON) (condition of type ESRAP-PARSE-ERROR)


NIL

####  Assign force-field parameters and minimize the energy of the molecule.

In [13]:
(cando:jostle *chemagg*)

NIL

In [14]:
(defparameter *ee* (energy:minimize *chemagg*))


An error occurred of type: SIMPLE-PROGRAM-ERROR: 
  Chiral center (ATOM(:C_0[residue(NIL)@0x11cdaac88]bonds[4])) with configuration settings[UndefinedConfiguration/ChiralCenter] doesn't have its configuration set

serious-condition backtrace:
0: (ERROR CORE:SIMPLE-PROGRAM-ERROR :FORMAT-CONTROL "Chiral center (ATOM(:C_0[residue(NIL)@0x11cdaac88]bonds[4])) with configuration settings[UndefinedConfiguration/ChiralCenter] doesn't have its configuration set" :FORMAT-ARGUMENTS NIL)
1: (APPLY #<FUNCTION ERROR> CORE:SIMPLE-PROGRAM-ERROR :FORMAT-CONTROL "Chiral center (ATOM(:C_0[residue(NIL)@0x11cdaac88]bonds[4])) with configuration settings[UndefinedConfiguration/ChiralCenter] doesn't have its configuration set" :FORMAT-ARGUMENTS NIL NIL)
2: (SIGNAL-SIMPLE-ERROR CORE:SIMPLE-PROGRAM-ERROR NIL "Chiral center (ATOM(:C_0[residue(NIL)@0x11cdaac88]bonds[4])) with configuration settings[UndefinedConfiguration/ChiralCenter] doesn't have its configuration set" NIL)
3: (MINIMIZE #<AGGREGATE NIL>)
4: (L

NIL

#### Visualize the minimized structure and demonstrate that the force field parameters lead to the construction of a chemically

#### reasonable structure.

In [15]:
(show *chemagg*)

#<NGLV:NGLWIDGET>

In [27]:
(cando::show-aggregate *chemagg*)

A Jupyter Widget

In [28]:
(:= *bcc* (charges:calculate-bcc-corrections *chemagg*))

rings NIL
rings NIL
rings NIL
rings NIL
rings NIL
rings NIL


*BCC*

In [29]:
 (:= *order* (charges:write-sqm-calculation (open "/tmp/sqm-input.txt" :direction :output) *chemagg*))

*ORDER*

In [30]:
(ext:vfork-execvp (list (namestring (translate-logical-pathname #P"amber:bin;sqm"))
                             "-O"
                             "-i" "/tmp/sqm-input.txt"
                             "-o" "/tmp/sqm-output.out"))

0

In [31]:
(:= *am1* (charges:read-am1-charges "/tmp/sqm-output.out" *order*))

*AM1*

In [32]:
(maphash (lambda (atom charge) 
                 (format t "atom ~a charge ~a~%" atom charge)) *am1*)

atom #<ATOM :HH32/:H> charge 0.085d0
atom #<ATOM :HH32/:H> charge 0.10300000000000001d0
atom #<ATOM :HH33/:H> charge 0.093d0
atom #<ATOM :HH31/:H> charge 0.10400000000000001d0
atom #<ATOM :NH/:N> charge -0.376d0
atom #<ATOM :C/:C> charge 0.301d0
atom #<ATOM :H/:H> charge 0.137d0
atom #<ATOM :CH3/:C> charge -0.079d0
atom #<ATOM :CH3/:C> charge -0.22d0
atom #<ATOM :HH32/:H> charge 0.07100000000000001d0
atom #<ATOM :C_0/:C> charge 0.006d0
atom #<ATOM :HH/:H> charge 0.227d0
atom #<ATOM :HH31/:H> charge 0.117d0
atom #<ATOM :NH/:N> charge -0.363d0
atom #<ATOM :O/:O> charge -0.38d0
atom #<ATOM :CH3/:C> charge -0.24200000000000002d0
atom #<ATOM :C/:C> charge 0.274d0
atom #<ATOM :HH33/:H> charge 0.117d0
atom #<ATOM :HH33/:H> charge 0.067d0
atom #<ATOM :HH31/:H> charge 0.094d0
atom #<ATOM :HH/:H> charge 0.23500000000000001d0
atom #<ATOM :O/:O> charge -0.371d0


NIL

In [33]:
(maphash (lambda (atom charge) 
                 (format t "atom ~a charge ~a~%" atom charge)) *bcc*)

atom #<ATOM :HH32/:H> charge 0.0393
atom #<ATOM :HH32/:H> charge 0.0393
atom #<ATOM :HH33/:H> charge 0.0393
atom #<ATOM :HH31/:H> charge 0.0393
atom #<ATOM :NH/:N> charge -0.0466
atom #<ATOM :C/:C> charge 0.1891
atom #<ATOM :H/:H> charge 0.0393
atom #<ATOM :CH3/:C> charge 0.27609998
atom #<ATOM :CH3/:C> charge 0.1179
atom #<ATOM :HH32/:H> charge 0.0393
atom #<ATOM :C_0/:C> charge 0.1475
atom #<ATOM :HH/:H> charge -0.2048
atom #<ATOM :HH31/:H> charge 0.0393
atom #<ATOM :NH/:N> charge -0.0466
atom #<ATOM :O/:O> charge 0.2391
atom #<ATOM :CH3/:C> charge 0.0679
atom #<ATOM :C/:C> charge 0.1891
atom #<ATOM :HH33/:H> charge 0.0393
atom #<ATOM :HH33/:H> charge 0.0393
atom #<ATOM :HH31/:H> charge 0.0393
atom #<ATOM :HH/:H> charge -0.2048
atom #<ATOM :O/:O> charge 0.2391


NIL

In [26]:
(maphash (lambda (atom charge) 
                 (format t "atom ~a charge ~a~%" atom charge)) *am1*)

atom #<ATOM :NH/:N> charge -0.377d0
atom #<ATOM :HH/:H> charge 0.23500000000000001d0
atom #<ATOM :C_0/:C> charge 0.006d0
atom #<ATOM :NH/:N> charge -0.364d0
atom #<ATOM :HH33/:H> charge 0.092d0
atom #<ATOM :HH32/:H> charge 0.07d0
atom #<ATOM :O/:O> charge -0.371d0
atom #<ATOM :HH/:H> charge 0.227d0
atom #<ATOM :HH32/:H> charge 0.10300000000000001d0
atom #<ATOM :H/:H> charge 0.137d0
atom #<ATOM :HH31/:H> charge 0.094d0
atom #<ATOM :HH33/:H> charge 0.117d0
atom #<ATOM :CH3/:C> charge -0.079d0
atom #<ATOM :CH3/:C> charge -0.24100000000000002d0
atom #<ATOM :O/:O> charge -0.38d0
atom #<ATOM :HH31/:H> charge 0.117d0
atom #<ATOM :CH3/:C> charge -0.22d0
atom #<ATOM :C/:C> charge 0.275d0
atom #<ATOM :HH31/:H> charge 0.10400000000000001d0
atom #<ATOM :C/:C> charge 0.302d0
atom #<ATOM :HH32/:H> charge 0.084d0
atom #<ATOM :HH33/:H> charge 0.068d0


NIL

In [34]:
(charges:calculate-am1-bcc-charges *chemagg*)

rings NIL
rings NIL
rings NIL
rings NIL
rings NIL
rings NIL


NIL

In [7]:
(save-mol2 *chemagg* "alanine.mol2")


An error occurred of type: UNBOUND-VARIABLE: 
  The variable *CHEMAGG* is unbound.

serious-condition backtrace:
0: (ERROR UNBOUND-VARIABLE :NAME *CHEMAGG*)
1: (LAMBDA)



NIL

#### Assign atom types and change one of them for alanine atom type definition.

In [37]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

#### Save the topology file and coordinate file.

In [8]:
(leap.topology:save-amber-parm-format *chemagg* "chemagg-new.top" "chemagg-new.crd" :assign-types nil)


An error occurred of type: UNBOUND-VARIABLE: 
  The variable *CHEMAGG* is unbound.

serious-condition backtrace:
0: (ERROR UNBOUND-VARIABLE :NAME *CHEMAGG*)
1: (LAMBDA)



NIL

## Create topology and coordinate file for Alanine and water atoms.

#### Load Tip3pfbox water box.

In [11]:
(load-off "tip3pfbox.off")

(:TIP3PFBOX)

#### Make agggregate of the water box.

In [12]:
(print ~tip3pfbox)


#<AGGREGATE NIL> 

#<AGGREGATE NIL>

In [13]:
(defparameter *solvent* ~tip3pfbox)

*SOLVENT*

In [14]:
(show *solvent*)

#<NGLV:NGLWIDGET>

#### Assign the atom types of the force field to the water box.

In [15]:
(assign-atom-types *solvent*)

NIL

#### Add Tip3p water molecules to the alanine.

In [16]:
(:= *chemagg* (load-mol2 "alanine_leap.mol2"))

*CHEMAGG*

In [17]:
(show *chemagg*)

#<NGLV:NGLWIDGET>

In [19]:
(leap.solvate::tool-solvate-and-shell *chemagg* *solvent* '(10.0 10.0 10.0))

Total bounding box for atom centers:   25.25  27.65  23.02
There are 1728 solvent molecules and 22 solute atoms
Completed  [**************************************************] ETC: --


NIL

In [20]:
(apropos "solvate")

SOLVATE-BOX  Function
LEAP::SOLVATE-OCT  Function
LEAP::SOLVATE-SHELL  Function
LEAP.SOLVATE:TOOL-SOLVATE-AND-SHELL  Function


NIL

In [40]:
(solvate-box *chemagg* *solvent* 20.0)

Total bounding box for atom centers:   50.93  45.00  43.99
There are 5832 solvent molecules and 22 solute atoms
Completed  [**************************************************] ETC: --

NIL

In [18]:
(leap::solvate-oct *chemagg* *solvent* 20.0)

Total bounding box for atom centers:   50.93  50.93  50.93
(box expansion for 'iso' is 31.05357719972584d0 )
There are 5832 solvent molecules and 22 solute atoms
Completed  [**************************************************] ETC: --

NIL

In [49]:
(leap::solvate-shell *chemagg* *solvent* 20.0)

Total bounding box for atom centers:   50.93  45.00  43.99
There are 5832 solvent molecules and 22 solute atoms
Completed  [**************************************************] ETC: --

NIL

In [25]:
(show *chemagg*)

#<NGLV:NGLWIDGET>

#### Save the topology and coordinate file. New aggregate contains water molecules and alanine.

In [26]:
(assign-atom-types *chemagg*)
(chem:map-atoms 'nil (lambda (a) (if (eq (chem:get-type a) :CT)
                                     (if (eq (chem:get-name a) :C)
                                         (chem:set-type a :CX)))) *chemagg*)

NIL

In [27]:
(:= *ef* (leap.topology:save-amber-parm-format *chemagg* "chemagg_water.top" "chemagg_water.crd" :assign-types nil))

Constructing energy function
Writing to /Users/shiho/Development/cando/extensions/cando/src/tests/alanine-demo/chemagg_water.top
Starting prepare-amber-energy-dihedral
Ordering i1,i2,i3,i4 prepare-amber-energy-dihedral
1-4 interactions prepare-amber-energy-dihedral
Counting w and w/o water prepare-amber-energy-dihedral
Extracting prepare-amber-energy-dihedral
Saving     [******************************************] ETC: --


*EF*

In [10]:
(show *chemagg*)

#<NGLV:NGLWIDGET>

#### Calculate each energy parameters.

In [31]:
(chem:get-energy (chem:get-stretch-component *e*))

1720.1189821006878d0

In [32]:
(chem:get-energy (chem:get-angle-component *e*))

0.30171081698364277d0

In [33]:
(chem:get-energy (chem:get-dihedral-component *e*))

5.456751705606037d0

In [34]:
(chem:get-energy (chem:get-nonbond-component *e*))

658.8024508583202d0

In [34]:
(apropos "superpose")

CHEM:ADD-SUPERPOSE-ATOM  Function
CHEM:CLEAR-SUPERPOSE-ATOMS  Function
CHEM:SET-FINAL-COORDINATES-AS-FIXED-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-FINAL-COORDINATES-AS-MOVEABLE-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-SUPERPOSABLE-COORDINATES-AS-FIXED-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SET-SUPERPOSABLE-COORDINATES-AS-MOVEABLE-WITHIN-SUPERPOSE-ENGINE  Function
CHEM:SUPERPOSE  Function
CANDO::SUPERPOSE-AGAINST-TEMPLATE  Function
CANDO::SUPERPOSE-ALL  Function
CHEM:SUPERPOSE-ALL-HEAVY-ATOMS  Function
CHEM:SUPERPOSE-ENGINE
CANDO::SUPERPOSE-ONE  Function
CHEM:SUPERPOSE-SELECTED-ATOMS
CHEM::SUPERPOSER


NIL

In [88]:
(:= w (show *chemagg*)) w

A Jupyter Widget

In [93]:
(nglv::picked w)

(("atom" ("index" . 20) ("residueIndex" . 2) ("resname" . "NIL_3")
  ("x" . 216.329) ("y" . 322.549) ("z" . -0.62) ("element" . "H")
  ("chainname" . "A") ("resno" . 3) ("serial" . 21) ("vdw" . 1.1)
  ("covalent" . 0.31) ("hetero" . 1) ("bfactor" . 0) ("altloc" . "")
  ("atomname" . "HH33")))

In [94]:
(defun distance (p1 p2) (geom:vlength (geom:v- p1 p2)))

DISTANCE

In [95]:
(:= *a* (let* ((picked (nglv::picked w))
       (atom (cdr (assoc "atom" picked :test #'string=)))
       (x (cdr (assoc "x" atom :test #'string=)))
       (y (cdr (assoc "y" atom :test #'string=)))
       (z (cdr (assoc "z" atom :test #'string=)))
       (pos (geom:vec x y z))
        resulta)
     (chem:map-atoms nil (lambda (a) 
                                 (when (< (distance pos (chem:get-position a)) 0.01) 
                                     (setf resulta a))) *chemagg*)
              resulta))

*A*

In [96]:
*a*

#<ATOM :HH33/:H>

In [97]:
(:= *p* (core:make-cxx-object 'chem:chem-info))

*P*

In [98]:
(chem:compile-smarts *p* "CC")

T

In [99]:
(chem:matches *p* *a*)

NIL

In [20]:
(:= *ala-leap* (load-mol2 "alanine_leap.mol2"))

*ALA-LEAP*

In [21]:
(cando::show-aggregate *ala-leap*)

A Jupyter Widget

In [12]:
(chem:map-atoms 'nil (lambda (a) (format t "type ~a name ~a~% " (chem:get-type a) (chem:get-name a)))
                
                             *ala-leap*)

type HC name H1
 type CT name CH3
 type HC name H2
 type HC name H3
 type C name C
 type O name O
 type N name N
 type H name H
 type CX name CA
 type H1 name HA
 type CT name CB
 type HC name HB1
 type HC name HB2
 type HC name HB3
 type C name C
 type O name O
 type N name N
 type H name H
 type CT name CH3
 type H1 name HH31
 type H1 name HH32
 type H1 name HH33
 

NIL

In [22]:
(leap.topology:save-amber-parm-format *ala-leap* "alanine_cando.top" "alanine_cando.crd" :assign-types nil)

atom-name-vector -> #(:H1 :CH3 :H2 :H3 :C :O :N :H :CA :HA :CB :HB1 :HB2 :HB3
                      :C :O :N :H :CH3 :HH31 :HH32 :HH33)
atom-type-vector -> #(:HC :CT :HC :HC :C :O :N :H :CX :H1 :CT :HC :HC :HC :C :O
                      :N :H :CT :H1 :H1 :H1)
type-index-vector -> #(24 13 24 24 0 50 41 23 17 25 13 24 24 24 0 50 41 23 13
                       25 25 25)
atom-name-vector -> #(:H1 :CH3 :H2 :H3 :C :O :N :H :CA :HA :CB :HB1 :HB2 :HB3
                      :C :O :N :H :CH3 :HH31 :HH32 :HH33)
atom-type-vector -> #(:HC :CT :HC :HC :C :O :N :H :CX :H1 :CT :HC :HC :HC :C :O
                      :N :H :CT :H1 :H1 :H1)
type-index-vector -> #(24 13 24 24 0 50 41 23 17 25 13 24 24 24 0 50 41 23 13
                       25 25 25)


T

In [42]:
(:= *ala-pdb* (load-pdb "ace-ala-nme.pdb"))

popped residue: #<PDB-RESIDUE ACE>
popped residue: #<PDB-RESIDUE ALA>
popped residue: #<PDB-RESIDUE NME>
Built 3 missing hydrogens


SIMPLE-WARNING: 
Loaded atom (1 H1 ACE NIL 1 NIL) but amber form #<TOPOLOGY  :name :ACE @0x122212e48> does not recognize it
SIMPLE-WARNING: 
Loaded atom (3 H2 ACE NIL 1 NIL) but amber form #<TOPOLOGY  :name :ACE @0x122212e48> does not recognize it
SIMPLE-WARNING: 
Loaded atom (4 H3 ACE NIL 1 NIL) but amber form #<TOPOLOGY  :name :ACE @0x122212e48> does not recognize it


*ALA-PDB*

In [25]:
(cando::show-aggregate *ala-pdb*)

A Jupyter Widget

In [26]:
(chem:map-atoms 'nil (lambda (a) (format t "type ~a name ~a~% " (chem:get-type a) (chem:get-name a)))
                
                             *ala-leap*)

type HC name H1
 type CT name CH3
 type HC name H2
 type HC name H3
 type C name C
 type O name O
 type N name N
 type H name H
 type CX name CA
 type H1 name HA
 type CT name CB
 type HC name HB1
 type HC name HB2
 type HC name HB3
 type C name C
 type O name O
 type N name N
 type H name H
 type CT name CH3
 type H1 name HH31
 type H1 name HH32
 type H1 name HH33
 

NIL

In [27]:
(leap.topology:save-amber-parm-format *ala-pdb* "alanine_pdb.top" "alanine_pdb.crd" :assign-types nil)

T

In [8]:
(apropos "read-amber")

CHEM:MAKE-READ-AMBER-PARAMETERS  Function
LEAP.TOPOLOGY::READ-AMBER-COORDINATE-FILE  Function
CHEM:READ-AMBER-PARAMETERS
LEAP.TOPOLOGY::READ-AMBER-PARM-FORMAT  Function


NIL

In [43]:
(:= *ef* (leap.topology::read-amber-parm-format "chemagg_no_water.top"))

Unknown flag %FLAG HBOND_ACOEF                                                               
Unknown flag %FLAG HBOND_BCOEF                                                               
Unknown flag %FLAG HBCUT                                                                     
Unknown flag %FLAG TREE_CHAIN_CLASSIFICATION                                                 
Unknown flag %FLAG JOIN_ARRAY                                                                
Unknown flag %FLAG IROTAT                                                                    
Unknown flag %FLAG RADIUS_SET                                                                
Unknown flag %FLAG RADII                                                                     
Unknown flag %FLAG SCREEN                                                                    


*EF*

In [ ]:
(:= *avec* (leap.topology::generate-aggregate-for-energy-function *ef*))

In [44]:
(:= *water-ef* (leap.topology::read-amber-parm-format "chemagg_water.top"))

Unknown flag %FLAG HBOND_ACOEF                                                               
Unknown flag %FLAG HBOND_BCOEF                                                               
Unknown flag %FLAG HBCUT                                                                     
Unknown flag %FLAG TREE_CHAIN_CLASSIFICATION                                                 
Unknown flag %FLAG JOIN_ARRAY                                                                
Unknown flag %FLAG IROTAT                                                                    
Unknown flag %FLAG BOX_DIMENSIONS                                                            
Unknown flag %FLAG RADIUS_SET                                                                
Unknown flag %FLAG RADII                                                                     
Unknown flag %FLAG SCREEN                                                                    


*WATER-EF*

In [8]:
(:= *pcsk9* (load-pdb "~/Desktop/2p4e.pdb"))


Creating molecule with name: P
Creating molecule with name: A




There are 2 unbuilt heavy atoms - not building hydrogens


*PCSK9*

In [39]:
(:= *compound1* (load-mol2 "~/Desktop/24MarIresult.mol2"))

*COMPOUND1*

In [14]:
(:= *v* (show *pcsk9*)) *v*


An error occurred of type: SIMPLE-PROGRAM-ERROR: 
  You must implement a test for: SAPBondedToPrevious

serious-condition backtrace:
0: (ERROR CORE:SIMPLE-PROGRAM-ERROR :FORMAT-CONTROL "You must implement a test for: SAPBondedToPrevious" :FORMAT-ARGUMENTS NIL)
1: (APPLY #<FUNCTION ERROR> CORE:SIMPLE-PROGRAM-ERROR :FORMAT-CONTROL "You must implement a test for: SAPBondedToPrevious" :FORMAT-ARGUMENTS NIL NIL)
2: (SIGNAL-SIMPLE-ERROR CORE:SIMPLE-PROGRAM-ERROR NIL "You must implement a test for: SAPBondedToPrevious" NIL)
3: (AGGREGATE-AS-MOL2-STRING #<AGGREGATE :2p4e> T)
4: ((METHOD GET-STRUCTURE-STRING ((CANDO-STRUCTURE))) #<CANDO-JUPYTER:CANDO-STRUCTURE>)
5: (_%LOAD-DATA^NGLV^((NGLWIDGET #<VA-LIST: (#<NGLV:NGLWIDGET> #<CANDO-JUPYTER:CANDO-STRUCTURE>)> NIL)
6: (APPLY NGLV::%LOAD-DATA #<NGLV:NGLWIDGET> #<CANDO-JUPYTER:CANDO-STRUCTURE> NIL)
7: (_ADD-STRUCTURE^NGLV^((NGLWIDGET #<VA-LIST: (#<NGLV:NGLWIDGET> #<CANDO-JUPYTER:CANDO-STRUCTURE>)> NIL)
8: (MAKE-NGLWIDGET :STRUCTURE #<CANDO-JUPYTER

NIL

In [20]:
(chem:add-matter *pcsk9* *compound*)

NIL

In [35]:
(assign-atom-types *compound*)

NIL

In [40]:
(show *compound1*)


An error occurred of type: SIMPLE-PROGRAM-ERROR: 
  You must implement a test for: SAPBondedToPrevious

serious-condition backtrace:
0: (ERROR CORE:SIMPLE-PROGRAM-ERROR :FORMAT-CONTROL "You must implement a test for: SAPBondedToPrevious" :FORMAT-ARGUMENTS NIL)
1: (APPLY #<FUNCTION ERROR> CORE:SIMPLE-PROGRAM-ERROR :FORMAT-CONTROL "You must implement a test for: SAPBondedToPrevious" :FORMAT-ARGUMENTS NIL NIL)
2: (SIGNAL-SIMPLE-ERROR CORE:SIMPLE-PROGRAM-ERROR NIL "You must implement a test for: SAPBondedToPrevious" NIL)
3: (AGGREGATE-AS-MOL2-STRING #<AGGREGATE :24MarIresult.pdb> T)
4: ((METHOD GET-STRUCTURE-STRING ((CANDO-STRUCTURE))) #<CANDO-JUPYTER:CANDO-STRUCTURE>)
5: (_%LOAD-DATA^NGLV^((NGLWIDGET #<VA-LIST: (#<NGLV:NGLWIDGET> #<CANDO-JUPYTER:CANDO-STRUCTURE>)> NIL)
6: (APPLY NGLV::%LOAD-DATA #<NGLV:NGLWIDGET> #<CANDO-JUPYTER:CANDO-STRUCTURE> NIL)
7: (_ADD-STRUCTURE^NGLV^((NGLWIDGET #<VA-LIST: (#<NGLV:NGLWIDGET> #<CANDO-JUPYTER:CANDO-STRUCTURE>)> NIL)
8: (MAKE-NGLWIDGET :STRUCTURE #<C

NIL